In [ ]:
# cfg customising
# classes=число_классов
# max_batches=classes*2000, но не меньше числа изображений в обучающей выборке
# steps to 80% and 90% of max_batches, f.e. steps=4800,5400 
# width=416 height=416 or any value multiple of 32
# есть конфиги-блоки 3х слоев [yolo], в них установить classes=число_классов
# перед конфигами-блоками 3х слоев [yolo] есть по одному конфигу-блоку [convolution] - в них установить filters=(classes + 5)x3 по формуле

In [1]:
import shutil
import os
from IPython.display import clear_output 
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
import datetime
import string
import glob
import math
import random
import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection
from google.colab import drive

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 13.98 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (10341/10341), done.


In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [4]:
# make darknet (build)
!make
clear_output()

In [5]:
%cd /content

/content


In [7]:
# https://drive.google.com/file/d/1DXfSbuLLQSWduoJDKEr5Hu3qpqquk-_P/view?usp=sharing

In [ ]:
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Share/'
!unzip /content/cells-y4t.zip -d /content/data  

In [9]:
# dont forget to upload data.zip to /content via gui
# download url https://drive.google.com/file/d/1DXfSbuLLQSWduoJDKEr5Hu3qpqquk-_P/view?usp=sharing
# file structure
# /content------
#  |___/data----
#       |___/obj
#       |___ ....
#  |___/darknet-
#  |___data.zip

#%cd /content
!unzip cells-y4t.zip -d /content/data 
clear_output()

In [ ]:
# training from scratch / weights saved in /content/data/backup - best weights on validation and last weights from each n iterations
!./darknet/darknet detector train ./data/obj.data ./data/yolov4-tiny-3l.cfg ./data/yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.
 4741: 1.349699, 1.240399 avg loss, 0.002610 rate, 2.259449 seconds, 303424 images, 0.842881 hours left
Loaded: 0.000066 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.881316), count: 7, class_loss = 0.269789, iou_loss = 0.732187, total_loss = 1.001976 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.847297), count: 23, class_loss = 1.222208, iou_loss = 5.780512, total_loss = 7.002720 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 44 Avg (IOU: 0.678017), count: 5, class_loss = 1.013428, iou_loss = 11.634327, total_loss = 12.647756 
 total_bbox = 3628051, rewritten_bbox = 0.120423 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.888338), count: 2, class_loss = 0.045234, iou_loss = 0.177250, total_loss = 0.222485 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.844143), co

In [ ]:
# continue to train best
from google.colab import files
files.download('/content/data/backup/yolov4-tiny-3l_1000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_2000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_3000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_4000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_5000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_6000.weights')
files.download('/content/data/backup/yolov4-tiny-3l_best.weights')
files.download('/content/data/backup/yolov4-tiny-3l_final.weights')
files.download('/content/data/backup/yolov4-tiny-3l_last.weights')
# !./darknet/darknet detector train ./data/obj.data ./data/yolov4-tiny-3l.cfg ./data/backup/yolov4-tiny-3l_last.weights -dont_show -map

FileNotFoundError: ignored

In [ ]:
# continue to train fine tuned
files.download('/content/data/backup/yolov4-tiny-3l_best.weights')
#!./darknet/darknet detector train ./data/obj.data ./data/yolov4-tiny-3l.cfg ./data/backup/yolov4-tiny-3l_fine_tuned.weights -dont_show -map

In [ ]:
def imShow(path):
    %matplotlib inline
    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
!rm -rf /content/data/results
!mkdir /content/data/results

Platelets = []
RBC = []
WBC = []

# save results to /content/results
for filename in os.listdir("/content/data/obj"):
  if filename.endswith("jpg"):
    !./darknet/darknet detector test ./data/obj.data ./data/yolov4-tiny-3l.cfg ./data/backup/yolov4-tiny-3l_last.weights ./data/obj/{filename} -dont_show -ext_output | tee pred.txt
    a_file = open("pred.txt", "r")
    lines = a_file.readlines()
    a_file.close()
    last_lines = lines[-6:]
    img = cv2.imread("/content/data/obj/"+filename)
    #cv2_imshow(img)
    res = []
    for line in last_lines:
      spl = line.split()
      try:
        cords = {"class":spl[0][:-1], "conf_value":spl[1][:-1], "left_x":int(spl[3]), "top_y":int(spl[5]), 'width':int(spl[7]), 'height':int(spl[9][:-1])}
        res.append(cords)
      except:
        continue
     
      
    for box in res:
      x = box['left_x']
      y = box['top_y']
      w = box['width']
      h = box['height']
      crop_img = img[y:y+h, x:x+w]
      apps = {
              'Platelets':Platelets.append,
              'RBC':RBC.append,
              'WBC':WBC.append
             }
      #cv2_imshow(crop_img)
      apps[box['class']](crop_img)

    #os.remove("pred.txt")
    f = open(filename+".txt", "w")
    for line in last_lines:
      f.write(line)
    f.close()
    shutil.move("/content/predictions.jpg", "/content/data/results/"+filename)
    shutil.move("/content/"+filename+".txt", "/content/data/results/"+filename+".txt")



In [ ]:
import shutil
shutil.make_archive('/content/predictions', 'zip', '/content/data/results')
shutil.make_archive('/content/blood', 'zip', '/content/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
files.download('/content/predictions.zip')

In [ ]:
for img in WBC:
  cv2_imshow(img)